In [1]:
import ee
import geopandas as gpd
import geemap
import xarray as xr
import wxee

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-nunezrimedio-tesina',opt_url='https://earthengine-highvolume.googleapis.com')

In [3]:
## Traigo geometría de Argentina y deselecciono la Patagonia

def get_buenos_aires_feature():
    # Load the FAO GAUL dataset
    provinces = ee.FeatureCollection("FAO/GAUL/2015/level1")

    # Define a list of provinces to exclude
    exclude_provinces = [
        'Tierra Del Fuego', 'Santa Cruz', 'Neuquen', 'Chubut', 'Rio Negro'
    ]

    # Create a filter to exclude these provinces
    exclude_filter = ee.Filter.Not(ee.Filter.inList('ADM1_NAME', exclude_provinces))

    # Filter for Argentina and exclude the specified provinces
    argentina_provinces = provinces.filter(ee.Filter.eq('ADM0_NAME', 'Argentina'))
    filtered_provinces = argentina_provinces.filter(exclude_filter)

    return filtered_provinces
    
buenos_aires_feature = get_buenos_aires_feature()
Argentina_geometry = buenos_aires_feature.geometry()


# Get the Buenos Aires feature
buenos_aires_feature = get_buenos_aires_feature()
Argentina_geometry = buenos_aires_feature.geometry()

# Creo una grilla

fishnet = geemap.fishnet(
    Argentina_geometry,
    rows=50,
    cols=30
)

In [4]:
fishnet.size().getInfo()

964

In [5]:
# Create a Map
m = geemap.Map(center=[-34.61, -58.44], zoom=10)  # Center around Buenos Aires

# Add the Buenos Aires polygon to the map
m.addLayer(ee.Feature(fishnet.toList(fishnet.size()).get(345).getInfo()).geometry(), {}, "Argentina")

# Display the map
m

Map(center=[-34.61, -58.44], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

In [5]:
for i in range(963,fishnet.size().getInfo()):
    print(i)
    aoi = ee.Feature(fishnet.toList(fishnet.size()).get(i).getInfo()).geometry()
    dw_filtered = wxee.TimeSeries("GOOGLE/DYNAMICWORLD/V1").filterDate("2014", "2026").filterBounds(aoi).select("label")
    
    def clip_image(image):
        return image.clip(aoi)
    
    dw_filtered = dw_filtered.map(clip_image)
    
    monthly_ts = dw_filtered.aggregate_time("year", ee.Reducer.mode())
    
    ds = monthly_ts.wx.to_xarray(region=ee.Feature(aoi).geometry(), scale=30)
    ds = ds.fillna(-1)
    ds = ds.clip(min=-128,max=127)
    ds["label"] = ds["label"].astype("int8")
    ds["x"] = ds["x"].astype("float32")
    ds["y"] = ds["y"].astype("float32")
    ds["time"] = ds["time"].dt.strftime("%Y-%m")
    ds.to_netcdf(rf"D:\Factor-Data\Tesina\Tesina\New folder\array_{i}")

963


Requesting data:   0%|          | 0/9 [00:00<?, ?it/s]